In [99]:
import nltk as nl
import pandas as pd
import numpy as np
from PyDictionary import PyDictionary
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVR, SVR
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
import operator
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import KFold
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score

from nltk.sentiment import SentimentAnalyzer # sentiment analysis
from nltk.tokenize import RegexpTokenizer #tokenization
from nltk.tokenize import TweetTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer # sentiment analysis VADER
from nltk.tokenize import sent_tokenize #sentence tokenizer
from sklearn.naive_bayes import GaussianNB

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [52]:
dictionary=PyDictionary()

In [100]:
##read File
fileclick = 'stopclickbait.json'
# fileclick = 'train1703.jsonl'
fileclick

'stopclickbait.json'

In [101]:
 from nltk.corpus import words

In [102]:
#Store file in a variable
varfile = pd.read_json(fileclick, lines = True)
varfile


,id,postText,truthClass
0,1,Should I Get Bings,clickbait
1,2,Which TV Female Friend Group Do You Belong In,clickbait
2,3,"The New ""Star Wars: The Force Awakens"" Trailer...",clickbait
3,4,"This Vine Of New York On ""Celebrity Big Brothe...",clickbait
4,5,A Couple Did A Stunning Photo Shoot With Their...,clickbait
5,6,How To Flirt With Queer Girls Without Making A...,clickbait
6,7,32 Cute Things To Distract From Your Awkward T...,clickbait
7,8,If Disney Princesses Were From Florida,clickbait
8,9,What's A Quote Or Lyric That Best Describes Yo...,clickbait
9,10,Natalie Dormer And Sam Claflin Play A Game To ...,clickbait


In [103]:
# np.array([varfile['postText'] if varfile['id'] = 608310377143799808])
from nltk.tokenize import RegexpTokenizer
varpd = pd.DataFrame.from_dict(varfile)

print(varpd.postText.iloc[0])


##define functions to count number of characters and words. return -1 if there are no words/characters present
def numChar(content):
    if content is None:
        return -1
    elif not content:
        return -1
    elif type(content) is str:
        return len(content)
    else:
        
        for i in content :
            return len(i)
        
    
def numWords(content):
    tokenizer = RegexpTokenizer(r'\w+')
    if content is None:
        return -1
    elif not content:
        return -1
    elif type(content) is str:
        content_no_punc = tokenizer.tokenize(content)
        if(len(content_no_punc)==0):
            return -1
        else:
            return len(content_no_punc)
    else:
        content_no_punc = tokenizer.tokenize(content[0])
        if(len(content_no_punc)==0):
            return -1
        else:
            return len(content_no_punc)
        
##similar functions like above. but return 0 instead of -1. we need it for counting difference in characters/words
        
def numChar0(content):
    if content is None:
        return 0
    elif not content:
        return 0
    elif type(content) is str:
        return len(content)
    else:
        
        for i in content :
            return len(i)
        
    
def numWords0(content):
    tokenizer = RegexpTokenizer(r'\w+')
    if content is None:
        return 0
    elif not content:
        return 0
    elif type(content) is str:
        content_no_punc = tokenizer.tokenize(content)
        return len(content_no_punc)
    else:
        content_no_punc = tokenizer.tokenize(content[0])
        return len(content_no_punc)


Should I Get Bings


In [104]:
#Feature: Number of Characters
features1 = varpd[['id']].copy()


postlen = pd.DataFrame(columns = ["postlen"])
# titlelen = pd.DataFrame(columns = ["titlelen"])
# desclen = pd.DataFrame(columns = ["desclen"])
# keylen = pd.DataFrame(columns = ["keylen"])
# caplen = pd.DataFrame(columns = ["caplen"])
# paralen = pd.DataFrame(columns = ["paralen"])

for ind in range(len(varpd.index)):
    
    postlen = postlen.append(pd.DataFrame([numChar(varpd.iloc[ind]['postText'])], columns = ["postlen"] ), ignore_index = True)
#     titlelen = titlelen.append(pd.DataFrame([numChar(varpd.iloc[ind]['targetTitle'])], columns = ["titlelen"] ), ignore_index = True)
#     desclen = desclen.append(pd.DataFrame([numChar(varpd.iloc[ind]['targetDescription'])], columns = ["desclen"] ), ignore_index = True)
#     keylen = keylen.append(pd.DataFrame([numChar(varpd.iloc[ind]['targetKeywords'])], columns = ["keylen"] ), ignore_index = True)
#     caplen = caplen.append(pd.DataFrame([numChar(varpd.iloc[ind]['targetCaptions'])], columns = ["caplen"] ), ignore_index = True)
#     paralen = paralen.append(pd.DataFrame([numChar(varpd.iloc[ind]['targetParagraphs'])], columns = ["paralen"] ), ignore_index = True)
    
    
    
# features1  =  pd.concat([features1, postlen, titlelen, desclen, keylen, caplen, paralen], axis=1)
features1  =  pd.concat([features1, postlen], axis=1)
#features

In [105]:
#Dataframe used for counting features which involves measuring difference of words/character(notice the functions used)

features0 = varpd[['id']].copy()


postlen0 = pd.DataFrame(columns = ["postlen"])
# titlelen0 = pd.DataFrame(columns = ["titlelen"])
# desclen0 = pd.DataFrame(columns = ["desclen"])
# keylen0 = pd.DataFrame(columns = ["keylen"])
# caplen0 = pd.DataFrame(columns = ["caplen"])
# paralen0 = pd.DataFrame(columns = ["paralen"])

for ind in range(len(varpd.index)):
    
    postlen0 = postlen0.append(pd.DataFrame([numChar0(varpd.iloc[ind]['postText'])], columns = ["postlen"] ), ignore_index = True)
#     titlelen0 = titlelen0.append(pd.DataFrame([numChar0(varpd.iloc[ind]['targetTitle'])], columns = ["titlelen"] ), ignore_index = True)
#     desclen0 = desclen0.append(pd.DataFrame([numChar0(varpd.iloc[ind]['targetDescription'])], columns = ["desclen"] ), ignore_index = True)
#     keylen0 = keylen0.append(pd.DataFrame([numChar0(varpd.iloc[ind]['targetKeywords'])], columns = ["keylen"] ), ignore_index = True)
#     caplen0 = caplen0.append(pd.DataFrame([numChar0(varpd.iloc[ind]['targetCaptions'])], columns = ["caplen"] ), ignore_index = True)
#     paralen0 = paralen0.append(pd.DataFrame([numChar0(varpd.iloc[ind]['targetParagraphs'])], columns = ["paralen"] ), ignore_index = True)
    
    
    
# features0  =  pd.concat([features0, postlen0, titlelen0, desclen0, keylen0, caplen0, paralen0], axis=1)
features0  =  pd.concat([features0, postlen0], axis=1)


features20 = varpd[['id']].copy()
postword0 = pd.DataFrame(columns = ["postword"])
# titleword0 = pd.DataFrame(columns = ["titleword"])
# descword0 = pd.DataFrame(columns = ["descword"])
# keyword0 = pd.DataFrame(columns = ["keyword"])
# capword0 = pd.DataFrame(columns = ["capword"])
# paraword0 = pd.DataFrame(columns = ["paraword"])

for ind in range(len(varpd.index)):
    
    postword0 = postword0.append(pd.DataFrame([numWords0(varpd.iloc[ind]['postText'])], columns = ["postword"] ), ignore_index = True)
#     titleword0 = titleword0.append(pd.DataFrame([numWords0(varpd.iloc[ind]['targetTitle'])], columns = ["titleword"] ), ignore_index = True)
#     descword0 = descword0.append(pd.DataFrame([numWords0(varpd.iloc[ind]['targetDescription'])], columns = ["descword"] ), ignore_index = True)
#     keyword0 = keyword0.append(pd.DataFrame([numWords0(varpd.iloc[ind]['targetKeywords'])], columns = ["keyword"] ), ignore_index = True)
#     capword0 = capword0.append(pd.DataFrame([numWords0(varpd.iloc[ind]['targetCaptions'])], columns = ["capword"] ), ignore_index = True)
#     paraword0 = paraword0.append(pd.DataFrame([numWords0(varpd.iloc[ind]['targetParagraphs'])], columns = ["paraword"] ), ignore_index = True)
    
    
    
# features20  =  pd.concat([features20, postword0, titleword0, descword0, keyword0, capword0, paraword0], axis=1)
features20  =  pd.concat([features20, postword0], axis=1)

In [106]:
#Feature: Number of Words
features2 = varpd[['id']].copy()
postword = pd.DataFrame(columns = ["postword"])
# titleword = pd.DataFrame(columns = ["titleword"])
# descword = pd.DataFrame(columns = ["descword"])
# keyword = pd.DataFrame(columns = ["keyword"])
# capword = pd.DataFrame(columns = ["capword"])
# paraword = pd.DataFrame(columns = ["paraword"])

for ind in range(len(varpd.index)):
    
    postword = postword.append(pd.DataFrame([numWords(varpd.iloc[ind]['postText'])], columns = ["postword"] ), ignore_index = True)
#     titleword = titleword.append(pd.DataFrame([numWords(varpd.iloc[ind]['targetTitle'])], columns = ["titleword"] ), ignore_index = True)
#     descword = descword.append(pd.DataFrame([numWords(varpd.iloc[ind]['targetDescription'])], columns = ["descword"] ), ignore_index = True)
#     keyword = keyword.append(pd.DataFrame([numWords(varpd.iloc[ind]['targetKeywords'])], columns = ["keyword"] ), ignore_index = True)
#     capword = capword.append(pd.DataFrame([numWords(varpd.iloc[ind]['targetCaptions'])], columns = ["capword"] ), ignore_index = True)
#     paraword = paraword.append(pd.DataFrame([numWords(varpd.iloc[ind]['targetParagraphs'])], columns = ["paraword"] ), ignore_index = True)
    
    
    
# features2  =  pd.concat([features2, postword, titleword, descword, keyword, capword, paraword], axis=1)
features2  =  pd.concat([features2, postword], axis=1)
# features2

In [60]:
#Feature: Difference in Number of words
# Not suitable for just postTitles
features3 = varpd[['id']].copy()

for i in range(1,6):
    for j in range(i+1,7):
        temp = pd.DataFrame(abs(features20.iloc[:,i]-features20.iloc[:,j]), columns = ["wordDiff"+str(i)+str(j)])
        features3 = pd.concat([features3, temp], axis = 1)

        
# features3

In [61]:
#Feature: Difference in Number of Characters
# Not suitable for just postTitles
features4 = varpd[['id']].copy()

for i in range(1,6):
    for j in range(i+1,7):
        temp = pd.DataFrame(abs(features0.iloc[:,i]-features0.iloc[:,j]), columns = ["charDiff"+str(i)+str(j)])
        features4 = pd.concat([features4, temp], axis = 1)

# features4


In [62]:
# Feature: Ratio of number of words
# Not suitable for just postTitles
features5 = varpd[['id']].copy()

for i in range(1,6):
    for j in range(i+1,7):
        temp = pd.DataFrame(abs(features2.iloc[:,i]/features2.iloc[:,j]), columns = ["wordRatio"+str(i)+str(j)])
        temp.loc[features2.iloc[:,i] == -1, "wordRatio"+str(i)+str(j)] = -1
        temp.loc[features2.iloc[:,j] == -1, "wordRatio"+str(i)+str(j)] = -1
        features5 = pd.concat([features5, temp], axis = 1)

# features5

In [63]:
# Feature: Ratio of number of Characters
# Not suitable for just postTitles
features6 = varpd[['id']].copy()

for i in range(1,6):
    for j in range(i+1,7):
        temp = pd.DataFrame(abs(features1.iloc[:,i]/features1.iloc[:,j]), columns = ["charRatio"+str(i)+str(j)])
        temp.loc[features1.iloc[:,i] == -1, "charRatio"+str(i)+str(j)] = -1
        temp.loc[features1.iloc[:,j] == -1, "charRatio"+str(i)+str(j)] = -1
        features6 = pd.concat([features6, temp], axis = 1)

# features6

In [64]:
#Function: Common words Between Article Keywords and Others
# Not suitable for just postTitles

def commonWords(keyword, content):
    tokenizer = RegexpTokenizer(r'\w+')
    keyword_no_punc = tokenizer.tokenize(keyword)
    if(len(keyword_no_punc)==0):
        return 0
    elif (content is None):
        return 0
    elif (not content):
        return 0
    elif type(content) is str:
        content_no_punc = tokenizer.tokenize(content)
        return len(list(set(keyword_no_punc) & set(content_no_punc)))
    else:
        content_no_punc = tokenizer.tokenize(content[0])
        return len(list(set(keyword_no_punc) & set(content_no_punc)))


In [65]:
#Feature: Common words Between Article Keywords and Others
# Not suitable for just postTitles


features7 = varpd[['id']].copy()

commonpost = pd.DataFrame(columns = ["commonpost"])
commontitle = pd.DataFrame(columns = ["commontitle"])
commondesc = pd.DataFrame(columns = ["commondesc"])
commoncap = pd.DataFrame(columns = ["commoncap"])
commonpara = pd.DataFrame(columns = ["commonpara"])

for ind in range(len(varpd.index)):
    
    commonpost = commonpost.append(pd.DataFrame([commonWords(varpd.iloc[ind]['targetKeywords'], varpd.iloc[ind]['postText'])], columns = ["commonpost"] ), ignore_index = True)
    commontitle = commontitle.append(pd.DataFrame([commonWords(varpd.iloc[ind]['targetKeywords'], varpd.iloc[ind]['targetTitle'])], columns = ["commontitle"] ), ignore_index = True)
    commondesc = commondesc.append(pd.DataFrame([commonWords(varpd.iloc[ind]['targetKeywords'], varpd.iloc[ind]['targetDescription'])], columns = ["commondesc"] ), ignore_index = True)
    commoncap = commoncap.append(pd.DataFrame([commonWords(varpd.iloc[ind]['targetKeywords'], varpd.iloc[ind]['targetCaptions'])], columns = ["commoncap"] ), ignore_index = True)
    commonpara = commonpara.append(pd.DataFrame([commonWords(varpd.iloc[ind]['targetKeywords'], varpd.iloc[ind]['targetParagraphs'])], columns = ["commonpara"] ), ignore_index = True)
    
    
    
features7  =  pd.concat([features7, commonpost, commontitle, commondesc, commoncap, commonpara], axis=1)

In [66]:
# features7

In [67]:
#Feature: Presence of an image
# Not suitable for just postTitles

features8 = varpd[['id']].copy()
imagePresent = pd.DataFrame(1, index = np.arange(2459), columns = ["imagePresent"])

for i in range(len(varpd.index)):
    if not varfile.iloc[i,1]:
        imagePresent.loc[i, "imagePresent"] = 0
#np.where( varfile['postMedia'] )
#np.arange(2459)
#varfile.iloc[:,1]
features8  =  pd.concat([features8, imagePresent], axis=1)
# features8

In [68]:
#Function: Number of formal words
# Not suitable for just postTitles

def formalWords(content):
    c=0
    tokenizer = RegexpTokenizer(r'\w+')
    if content is None:
        return 0
    elif not content:
        return 0
    elif type(content) is str:
        content_no_punc = tokenizer.tokenize(content)
        for word in [x.lower() for x in content_no_punc]:
            if word in words.words():
                c=c+1
        return c
    else:
        content_no_punc = tokenizer.tokenize(content[0])
        for word in [x.lower() for x in content_no_punc]:
            if word in words.words():
                c=c+1
        return c

In [69]:
#Feature: Number of Formal Words
# features9 = varpd[['id']].copy()
# postform = pd.DataFrame(columns = ["postform"])
# titleform = pd.DataFrame(columns = ["titleform"])
# descform = pd.DataFrame(columns = ["descform"])
# keyform = pd.DataFrame(columns = ["keyform"])
# capform = pd.DataFrame(columns = ["capform"])
# paraform = pd.DataFrame(columns = ["paraform"])

# for ind in range(len(varpd.index)):
    
#     postform = postform.append(pd.DataFrame([formalWords(varpd.iloc[ind]['postText'])], columns = ["postform"] ), ignore_index = True)
#     titleform = titleform.append(pd.DataFrame([formalWords(varpd.iloc[ind]['targetTitle'])], columns = ["titleform"] ), ignore_index = True)
#     descform = descform.append(pd.DataFrame([formalWords(varpd.iloc[ind]['targetDescription'])], columns = ["descform"] ), ignore_index = True)
#     keyform = keyform.append(pd.DataFrame([formalWords(varpd.iloc[ind]['targetKeywords'])], columns = ["keyform"] ), ignore_index = True)
#     capform = capform.append(pd.DataFrame([formalWords(varpd.iloc[ind]['targetCaptions'])], columns = ["capform"] ), ignore_index = True)
#     paraform = paraform.append(pd.DataFrame([formalWords(varpd.iloc[ind]['targetParagraphs'])], columns = ["paraform"] ), ignore_index = True)
#     if(ind%10 == 0): 
#         print(ind)
    
    
# features9  =  pd.concat([features9, postform, titleform, descform, keyform, capform, paraform], axis=1)
# features9

In [70]:
# #Feature: Ratio of number of formal words to total number of words
# def formalRatio(formal, total):
#     if (total==-1):
#         return -1
#     else:
#         return abs(formal/total)


# features10 = varpd[['id']].copy()
# postratio = pd.DataFrame(columns = ["postratio"])
# titleratio = pd.DataFrame(columns = ["titleratio"])
# descratio = pd.DataFrame(columns = ["descratio"])
# keyratio = pd.DataFrame(columns = ["keyratio"])
# capratio = pd.DataFrame(columns = ["capratio"])
# pararatio = pd.DataFrame(columns = ["pararatio"])

# for ind in range(len(varpd.index)):
    
#     postratio = postratio.append(pd.DataFrame([formalRatio(features9.iloc[ind]['postform'], features2.iloc[ind]['postword'] )], columns = ["postratio"] ), ignore_index = True)
#     titleratio = titleratio.append(pd.DataFrame([formalRatio(features9.iloc[ind]['titleform'],features2.iloc[ind]['titleword'])], columns = ["titleratio"] ), ignore_index = True)
#     descratio = descratio.append(pd.DataFrame([formalRatio(features9.iloc[ind]['descform'],features2.iloc[ind]['descword'])], columns = ["descratio"] ), ignore_index = True)
#     keyratio = keyratio.append(pd.DataFrame([formalRatio(features9.iloc[ind]['keyform'],features2.iloc[ind]['keyword'])], columns = ["keyratio"] ), ignore_index = True)
#     capratio = capratio.append(pd.DataFrame([formalRatio(features9.iloc[ind]['capform'],features2.iloc[ind]['capword'])], columns = ["capratio"] ), ignore_index = True)
#     pararatio = pararatio.append(pd.DataFrame([formalRatio(features9.iloc[ind]['paraform'],features2.iloc[ind]['paraword'])], columns = ["pararatio"] ), ignore_index = True)
    
    
    
# features10  =  pd.concat([features10, postratio, titleratio, descratio, keyratio, capratio, pararatio], axis=1)
# features10

In [71]:
# twenty = fetch_20newsgroups()
# twenty.data
# Not suitable for just postTitles
raw_documents1 = varpd["postText"]
temp1 = pd.Series.tolist(raw_documents1)
temp4 = [item for items in temp1 for item in items]
raw_documents2 = varpd["targetParagraphs"]
temp2 = pd.Series.tolist(raw_documents2)
temp3 = [' '.join(item) for item in temp2]
vocab = temp4 + temp3

# newlist
tfidf = TfidfVectorizer().fit_transform(vocab, y = None)
# tfidf

In [72]:
# linear_kernel(tfidf[0:1], tfidf[2459:2459+1]).flatten()
# i

In [73]:
cosine_similarities = []
for i in range(int(len(vocab)/2)):
    cosine_similarities.append(linear_kernel(tfidf[i:i+1], tfidf[2459+i:2459+i+1]).flatten())
cosine_similarities = np.concatenate(cosine_similarities).ravel().tolist()

In [74]:
features12 = varpd[['id']].copy()
# Not suitable for just postTitles
tfidfcosine = pd.DataFrame(cosine_similarities, columns = ["tfidfcosine"])
features12 = pd.concat([features12, tfidfcosine], axis = 1) 
# features12 

In [107]:
finalFeaturesT = pd.merge(features1, features2,  on='id')
# finalFeaturesT = pd.merge(finalFeaturesT, features3,  on='id')
# finalFeaturesT = pd.merge(finalFeaturesT, features4,  on='id')
# finalFeaturesT = pd.merge(finalFeaturesT, features5,  on='id')
# finalFeaturesT = pd.merge(finalFeaturesT, features6,  on='id')
# finalFeaturesT = pd.merge(finalFeaturesT, features7,  on='id')
# finalFeaturesT = pd.merge(finalFeaturesT, features8,  on='id')
# finalFeaturesT = pd.merge(finalFeaturesT, features11,  on='id')
# finalFeaturesT = pd.merge(finalFeaturesT, features12,  on='id')
# finalFeaturesT = pd.merge(finalFeaturesT, features,  on='id')

# finalFeatures 

In [108]:
##Getting the Labels
# testLabelsFile = 'traintruth1703.jsonl'
Labels = varpd[['id', 'truthClass']].copy()

Labels.loc[Labels.iloc[:,1] == "no-clickbait", "truthClass"] = -1
Labels.loc[Labels.iloc[:,1] == "clickbait", "truthClass"] = 1
finalFeatures = pd.merge(finalFeaturesT, Labels[["id","truthClass"]], on = "id")

In [137]:
finalFeatures.loc[(finalFeatures['id'] <= 16001) & (finalFeatures['id'] >= 15991)]
finalFeatures.to_csv("features_nirmal_all_stopclickbait.csv", index=False, encoding='utf-8')

In [77]:
##Final Features with Truth Mean
finalFeaturesMSE = pd.merge(finalFeaturesT, Labels[["id","truthMean"]], on = "id")

In [78]:
def normalize(pd):
    featNorm = pd.copy()
    for k in range(len(pd.columns)):
        m = np.mean(pd.iloc[:,k])
        s = np.std(pd.iloc[:,k])
        featNorm.iloc[:,k] = (pd.iloc[:,k]-m)/s
    featNorm = featNorm.iloc[:,1:k]
    return(featNorm)

In [79]:
def getAccuracy(classifier, featuresel, numFeat, allfeatures, featurelist = finalFeatures):
    #Choose Classifier
    if(classifier == "SVM"):
        clf = SVC()
    elif(classifier == "Log"):
        clf = LogisticRegressionCV()
    elif(classifier == "RandomForest"):
        clf = RandomForestClassifier(criterion = 'entropy')
    elif(classifier == "XgBoost"):
        clf = GradientBoostingClassifier()
    elif(classifier == "NaiveBayes"):
        clf = GaussianNB()
    
    

        
    # select features we want
    x = [col for col in allfeatures if col in featurelist]
    x = allfeatures[x]

    
    featNorm = normalize(x)
    #print(featNorm)
    #print(featNorm)
    # Labels
    y = allfeatures.iloc[:,-1]
    
    if(featuresel== "chi"):
        X_new = SelectKBest(chi2, k=numFeat).fit_transform(featNorm, y)
    elif(featuresel == "f_classif"):
        X_new = SelectKBest(f_classif, k=numFeat).fit_transform(featNorm, y)
    elif(featuresel == "mutual_info"):
        X_new = SelectKBest(mutual_info_classif, k=numFeat).fit_transform(featNorm, y)
    else:
        X_new = featNorm
    #Classifier result
    scores = cross_val_score(clf, X_new, y, cv=10)
    accuracy = [scores.mean(), scores.std()]
    return(accuracy)
    

In [80]:
def useNirmalFunctionEfficiently(classifier, featuresel, numFeat,df):
    accuracy = getAccuracy(classifier, featuresel, numFeat, df, df.iloc[:,range(len(df.columns)-1)])
    return(accuracy)

In [81]:
classifiers = ["Log","SVM","RandomForest","XgBoost","NaiveBayes"]
nrFeats = [10,50,100,150,250,300,350,400,450, 500,550,600]
resultsposPatterns=pd.DataFrame(index = nrFeats)
for classif in classifiers:
    print(classif)
    accuracies=[]
    for nfeat in nrFeats:
        [acc1,acc2]=useNirmalFunctionEfficiently(classif,"f_classif",nfeat,finalFeatures)
        accuracies.append(acc1)
        print(nfeat)
    resultsposPatterns[classif]=accuracies


Log
10
50


ValueError: k should be >=0, <= n_features; got 100.Use k='all' to return all features.

In [ ]:
resultsposPatterns

In [82]:
# info_gain = getAccuracy("SVM", "none", 0,  finalFeatures, finalFeatures)

In [83]:
# info_gain = getAccuracy("Log",  "none", 0, finalFeatures, finalFeatures.iloc[:,range(len(finalFeatures.columns)-1)])

In [84]:
# info_gain = getAccuracy("RandomForest", "none", finalFeatures, finalFeatures.iloc[:,range(len(finalFeatures.columns)-1)])

In [85]:
# info_gain = getAccuracy("XgBoost", "none", finalFeatures, finalFeatures.iloc[:,range(len(finalFeatures.columns)-1)])

In [86]:
# info_gain = getAccuracy("Log",  "f_classif", 40, finalFeatures, finalFeatures.iloc[:,range(len(finalFeatures.columns)-1)])

In [87]:
# info_gain = getAccuracy("Log",  "f_classif", 60, finalFeatures, finalFeatures.iloc[:,range(len(finalFeatures.columns)-1)])

In [88]:
# info_gain = getAccuracy("Log",  "f_classif", 30, finalFeatures, finalFeatures.iloc[:,range(len(finalFeatures.columns)-1)])

In [89]:
# info_gain = getAccuracy("SVM",  "mutual_info", 500, finalFeatures, finalFeatures.iloc[:,range(len(finalFeatures.columns)-1)])

In [90]:
# from sklearn.linear_model import LogisticRegressionCV

# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score

# # X_train, X_test, y_train, y_test = train_test_split(featwithLab.iloc[:,1:91], featwithLab.iloc[:,91], test_size=0.4, random_state=0)

# clf = LogisticRegressionCV()
# scores = cross_val_score(clf, featwithLab.iloc[:,1:81], featwithLab.iloc[:,81], cv=10)
# # clf.fit(X_train, y_train)
# # clf.score(X_test, y_test)
# print("Accuracy: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [91]:
# from sklearn.ensemble import GradientBoostingClassifier

# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score

# # X_train, X_test, y_train, y_test = train_test_split(featwithLab.iloc[:,1:91], featwithLab.iloc[:,91], test_size=0.4, random_state=0)

# clf = GradientBoostingClassifier()
# scores = cross_val_score(clf, featwithLab.iloc[:,1:81], featwithLab.iloc[:,81], cv=10)
# # clf.fit(X_train, y_train)
# # clf.score(X_test, y_test)
# print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))

In [92]:
# from sklearn.ensemble import RandomForestClassifier
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score

# # X_train, X_test, y_train, y_test = train_test_split(featwithLab.iloc[:,1:91], featwithLab.iloc[:,91], test_size=0.4, random_state=0)

# clf = RandomForestClassifier(criterion = 'entropy')
# scores = cross_val_score(clf, featwithLab.iloc[:,1:81], featwithLab.iloc[:,81], cv=20)
# # clf.fit(X_train, y_train)
# # clf.score(X_test, y_test)
# print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))

In [93]:
# from sklearn.feature_selection import mutual_info_classif
# import operator

# info_gain = dict(zip(list(featwithLab.iloc[:,1:81]), mutual_info_classif( featwithLab.iloc[:,1:81], featwithLab.iloc[:,81])))


In [94]:
# c = 0
# for w in sorted(info_gain, key = info_gain.get, reverse = True):
#     c=c+1
#     print (w, info_gain[w])
#     if (c==20):
#         break

# Stemming and Word Cloud


In [95]:
from collections import Counter
from nltk.util import ngrams
from nltk.stem import *
from sklearn.feature_extraction.text import CountVectorizer # count unigrams
import re

In [96]:
#Retrieve words from postTitles
postTitles = varpd["postText"].tolist()

def stemmingPostTitles(postTitles,stem):
    tokenizer =  TweetTokenizer()
    if (stem=="porter"):
        stemmer = nl.PorterStemmer() # Porter Stemming
    elif (stem == "snowball"):
        stemmer = SnowballStemmer("english")
    elif (stem == "lancaster"):
        stemmer = LancasterStemmer()
    
    number_pattern = re.compile(r'\d+')
    corpus=[]
    for title in postTitles:
        title = title[0]
        
        title=title.lower()
        tokens = tokenizer.tokenize(title)
        stem_tokens = [stemmer.stem(t) for t in tokens ]
        stem_title = " ".join(stem_tokens)
        stem_title = number_pattern.sub('[N]', stem_title)
        corpus.append(stem_title)
    return(corpus)


In [97]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        #stopwords=stopwords,
        max_words=50,
        max_font_size=40, 
        scale=3,
        random_state=1 # chosen at random by flipping a coin; it was heads
    ).generate(str(data))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()


ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
from os import path
from scipy.misc import imread
import matplotlib.pyplot as plt
import random

from wordcloud import WordCloud, STOPWORDS

text = 'all your base are belong to us all of your base base base'
wordcloud = WordCloud(
                      relative_scaling = 1.0,
                      stopwords = {'to', 'of'} # set or space-separated string
                      ).generate(text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
addLabel = pd.merge(varpd, Labels[["id","truthClass"]], on = "id")
clickbait = addLabel.loc[addLabel["truthClass"]==1]
nonclickbait = addLabel.loc[addLabel["truthClass"]==-1]

In [ ]:
show_wordcloud(clickbait["postText"])

In [ ]:
show_wordcloud(nonclickbait["postText"])

In [ ]:
stemmedclickbait=stemmingPostTitles(clickbait["postText"].tolist(),"porter")
show_wordcloud(stemmedclickbait)

In [ ]:
nonstemmedclickbait=stemmingPostTitles(nonclickbait["postText"].tolist(),"porter")
show_wordcloud(nonstemmedclickbait)

# Analysis of Classifier results

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
# normalise features
featNorm = finalFeatures.copy()
for k in range(len(featNorm.columns)):
    m = np.mean(finalFeatures.iloc[:,k])
    s = np.std(finalFeatures.iloc[:,k])
    featNorm.iloc[:,k] = (finalFeatures.iloc[:,k]-m)/s
y = finalFeatures.iloc[:,-1]


#X_new = SelectKBest(f_classif, k=500).fit_transform(featNorm.iloc[:,1:len(featNorm.columns)-1], y)
X_train, X_test, y_train, y_test = train_test_split(featNorm.iloc[:,1:len(featNorm.columns)-1], finalFeatures.iloc[:,-1], test_size=0.2, random_state=0)
clf = LogisticRegressionCV()
# scores = cross_val_score(clf, featwithLab.iloc[:,1:81], featwithLab.iloc[:,81], cv=10)
clf.fit(X_train, y_train)
scores = clf.score(X_test, y_test)
print("Accuracy: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

misclassified = np.where(y_test != clf.predict(X_test))

In [98]:
len(finalFeatures.loc[finalFeatures["truthClass"]==1])/len(finalFeatures)

0.3098820658804392

In [99]:
# misclassified

In [100]:
mis = X_test.iloc[misclassified]

In [101]:
mis1 = mis.join(varpd, how= "inner")
# mis1

In [102]:
# pd.merge(mis1, Labels[["id","truthClass"]], on = "id").to_csv("misclassified.csv")

In [103]:
# finalFeatures

   # MSE
  
  

In [197]:
posPatterns = pd.read_csv("posPatterns_ALL_SVM.csv")
unigrams = pd.read_csv("unigrams_select250_Log.csv")
mixed = pd.read_csv("mixtureDF_AllFeatures_SVM.csv") 

posPatternClass = posPatterns.iloc[:,1:len(posPatterns.columns)]
unigramsClass = unigrams.iloc[:,1:len(unigrams.columns)]
mixedClass = mixed.iloc[:,1:len(mixed.columns)]

posPatternM = posPatterns.iloc[:,1:len(posPatterns.columns)-1]
unigramsM = unigrams.iloc[:,1:len(unigrams.columns)-1]
mixedM = mixed.iloc[:,1:len(mixed.columns)-1]

posPatternM = pd.merge(posPatternM, Labels[["id","truthMean"]], on = "id")
unigramsM = pd.merge(unigramsM, Labels[["id","truthMean"]], on = "id")
mixedM = pd.merge(mixedM, Labels[["id","truthMean"]], on = "id")

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [105]:


def computeMSE(finalFeaturesMSE, cross_val, featuresel, numFeat, regressor):
    
    ##set up cross validation
    cv_idx = KFold(n=len(varpd), n_folds=cross_val, shuffle=True, random_state=1)
    
    featNormMSE = normalize(finalFeaturesMSE)
    
    #only works with nd numpy arrays
    y = finalFeaturesMSE.truthMean.values
    X = featNormMSE.as_matrix(columns = featNormMSE.columns[1:len(featNormMSE.columns)-1])
    
    #feature selection
    if(featuresel== "chi"):
        X_new = SelectKBest(chi2, k=numFeat).fit_transform(X, y)
    elif(featuresel == "f_classif"):
        X_new = SelectKBest(f_classif, k=numFeat).fit_transform(X, y)
    elif(featuresel == "mutual_info"):
        X_new = SelectKBest(mutual_info_classif, k=numFeat).fit_transform(X, y)
    else:
        X_new = X
    print(X_new)
    #regression
    y_out = pd.DataFrame()
    scores = []
    count = 0
    for train_idx, val_idx in cv_idx:
            if(regressor == "LinearSVR"):
                clr = LinearSVR(C=0.001, loss='squared_epsilon_insensitive', dual=False, random_state=1)
            elif(regressor == "Log"):
                clr = LogisticRegression()
            elif(regressor == "XgBoost"):
                clr = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
            clr.fit(X_new[train_idx], y[train_idx])

            y_pred = clr.predict(X_new[val_idx])
            y_pred[y_pred < 0] = 0.0
            y_pred[y_pred > 1] = 1.0
#             y_out["c"+str(count)]=y_pred
#             count = count + 1
            
#     return(y_out.mean(axis=1))
            rmse = mean_squared_error(y[val_idx], y_pred)
            scores.append(rmse)

    m = np.mean(scores)
    s = np.std(scores)
    print("MSE: %0.3f (+/- %0.2f)" % (m, s))

In [106]:
def computeMixedAccuracy(posPatternClass, unigramsClass,  mixedClass, finalFeatures, finalFeaturesMSE, cross_val):
    
    ##set up cross validation
    cv_idx = KFold(n=len(varpd), n_folds=cross_val, shuffle=True, random_state=1)
    
    yclass = finalFeatures.iloc[:,-1]
    posPatternMMSE = normalize(posPatternClass)
    unigramsMMSE = normalize(unigramsClass)
    mixedMMSE = normalize(mixedClass)
    finalFeaturesNormMSE = normalize(finalFeatures)
    
    #only works with nd numpy arrays
    
    ycont = finalFeaturesMSE.truthMean.values
    yp = posPatternClass.truthClass.values
    yu = unigramsClass.truthClass.values
    ym = mixedClass.truthClass.values
    yf = finalFeatures.truthClass.values
    
    Xp = posPatternMMSE.as_matrix(columns = posPatternMMSE.columns[1:len(posPatternMMSE.columns)-1])
    Xu = unigramsMMSE.as_matrix(columns = unigramsMMSE.columns[1:len(unigramsMMSE.columns)-1])
    Xm = mixedMMSE.as_matrix(columns = mixedMMSE.columns[1:len(mixedMMSE.columns)-1])
    Xf = finalFeaturesNormMSE.as_matrix(columns = finalFeaturesNormMSE.columns[1:len(finalFeaturesNormMSE.columns)-1])
    

    Xp_new = SelectKBest(f_classif, k="all").fit_transform(Xp, yp)
    Xu_new = SelectKBest(f_classif, k=250).fit_transform(Xu, yu)
    Xm_new = SelectKBest(f_classif, k="all").fit_transform(Xm, ym)
    Xf_new = SelectKBest(f_classif, k="all").fit_transform(Xf, yf)
    
    
    y_out = pd.DataFrame()
    scores = []
    acc = []
    scoresM = []
    count = 0
    for train_idx, val_idx in cv_idx:
           
            clrp = SVC(probability = True)
            clru = LogisticRegression()
            clrm = SVC(probability = True)
            clrf = LogisticRegression()
                           
            clrp.fit(Xp_new[train_idx], yp[train_idx])
            clru.fit(Xu_new[train_idx], yu[train_idx])
            clrm.fit(Xm_new[train_idx], ym[train_idx])
            clrf.fit(Xf_new[train_idx], yf[train_idx])

            yp_pred = clrp.predict_proba(Xp_new[val_idx])[:,1]
            
            
            yu_pred = clru.predict_proba(Xu_new[val_idx])[:,1]
            
            
            ym_pred = clrm.predict_proba(Xm_new[val_idx])[:,1]
           
            
            yf_pred = clrf.predict_proba(Xf_new[val_idx])[:,1]
            
#             y_out["c"+str(count)]=y_pred
#             count = count + 1
            
            d= {'p':yp_pred.tolist(), 'u':yu_pred.tolist(), 'm':ym_pred.tolist(), 'f':yf_pred.tolist()}
            y_pred_final = pd.DataFrame(d)
            y_pred_final["mean"] = y_pred_final.mean(axis=1)
            y_pred_final["median"] = y_pred_final.iloc[:,0:4].median(axis=1)
            y_pred_final['class'] = np.where(y_pred_final['mean'] >= 0.5, 1,-1)
#             y_pred_final['class'] = np.where(y_pred_final['median'] >= 0.5, 1,-1)
            
            rmse = mean_squared_error(ycont[val_idx], y_pred_final["mean"])
            scores.append(rmse)
            
            rmsemed = mean_squared_error(ycont[val_idx], y_pred_final["median"])
            scoresM.append(rmsemed)
            
            accuracy = accuracy_score(yclass[val_idx], y_pred_final["class"])
            acc.append(accuracy)
    
#             rmse = mean_squared_error(y[val_idx], y_pred)
#             scores.append(rmse)

    mM = np.mean(scores)
    sM = np.std(scores)
    print("MSE: %0.3f (+/- %0.2f)" % (mM, sM))
#     return(y_pred_final)
    mA = np.mean(acc)
    sA = np.std(acc)
    print("Accuracy: %0.3f (+/- %0.2f)" % (mA, sA))
    
    mMed = np.mean(scoresM)
    sMed = np.std(scoresM)
    print("MSEmedian: %0.3f (+/- %0.2f)" % (mMed, sMed))

In [219]:
def computeMixedAccuracyLog(posPatternClass, unigramsClass,  mixedClass, finalFeatures, finalFeaturesMSE, cross_val,tdepth):
    
    ##set up cross validation
    cv_idx = KFold(n=len(varpd), n_folds=cross_val, shuffle=True, random_state=1)
    
    yclass = finalFeatures.iloc[:,-1]
    posPatternMMSE = normalize(posPatternClass)
    unigramsMMSE = normalize(unigramsClass)
    mixedMMSE = normalize(mixedClass)
    finalFeaturesNormMSE = normalize(finalFeatures)
    
    #only works with nd numpy arrays
    
    ycont = finalFeaturesMSE.truthMean.values
    yp = posPatternClass.truthClass.values
    yu = unigramsClass.truthClass.values
    ym = mixedClass.truthClass.values
    yf = finalFeatures.truthClass.values
    
    Xp = posPatternMMSE.as_matrix(columns = posPatternMMSE.columns)
    Xu = unigramsMMSE.as_matrix(columns = unigramsMMSE.columns)
    Xm = mixedMMSE.as_matrix(columns = mixedMMSE.columns)
    Xf = finalFeaturesNormMSE.as_matrix(columns = finalFeaturesNormMSE.columns)
    

    Xp_new = SelectKBest(f_classif, k="all").fit_transform(Xp, yp)
    Xu_new = SelectKBest(f_classif, k=250).fit_transform(Xu, yu)
    Xm_new = SelectKBest(f_classif, k="all").fit_transform(Xm, ym)
    Xf_new = SelectKBest(f_classif, k="all").fit_transform(Xf, yf)
    
    
    y_out = pd.DataFrame()
#     scores = []
    accM = []
    accS = []
#     scoresM = []
    count = 0
    for train_idx, val_idx in cv_idx:
           
            clrp = SVC(probability = True)
            clru = LogisticRegression()
            clrm = SVC(probability = True)
            clrf = LogisticRegression()
                           
            clrp.fit(Xp_new[train_idx], yp[train_idx])
            clru.fit(Xu_new[train_idx], yu[train_idx])
            clrm.fit(Xm_new[train_idx], ym[train_idx])
            clrf.fit(Xf_new[train_idx], yf[train_idx])

            yp_pred = clrp.predict_proba(Xp_new[val_idx])[:,1]
            yp_pred_train = clrp.predict_proba(Xp_new[train_idx])[:,1]
            
            
            yu_pred = clru.predict_proba(Xu_new[val_idx])[:,1]
            yu_pred_train = clru.predict_proba(Xu_new[train_idx])[:,1]
            
            ym_pred = clrm.predict_proba(Xm_new[val_idx])[:,1]
            ym_pred_train = clrm.predict_proba(Xm_new[train_idx])[:,1]
           
            
            yf_pred = clrf.predict_proba(Xf_new[val_idx])[:,1]
            yf_pred_train = clrf.predict_proba(Xf_new[train_idx])[:,1]
            
#             y_out["c"+str(count)]=y_pred
#             count = count + 1
            
            d1= {'p':yp_pred.tolist(), 'u':yu_pred.tolist(), 'm':ym_pred.tolist(), 'f':yf_pred.tolist(), 'y': yclass[val_idx].tolist(), 'ycont': ycont[val_idx].tolist()}
            d2= {'p':yp_pred_train.tolist(), 'u':yu_pred_train.tolist(), 'm':ym_pred_train.tolist(), 'f':yf_pred_train.tolist(), 'y': yclass[train_idx].tolist(), 'ycont': ycont[train_idx].tolist()}
            df1 = pd.DataFrame(d1)
            df2 = pd.DataFrame(d2)
            
            d= pd.concat([df1, df2])
            
            clf = RandomForestRegressor(max_depth=tdepth, random_state=0) #i used random forest
            
#             scores = cross_val_score(clf, d[["p", "u", "m", "f"]], d["y"], cv=10)
            #rand = pd.DataFrame(np.random.randn(2459, 2))

            #msk = np.random.rand(len(rand)) < 0.8

            #train = d[msk]
            X_train, y_train, ycont_train = df2[["p", "u", "m", "f"]], df2["y"], df2["ycont"]
            #test = d[~msk]
            X_test, y_test, ycont_test = df1[["p", "u", "m", "f"]], df1["y"], df1["ycont"]
#             X_train, X_test, y_train, y_test = train_test_split( d[["p", "u", "m", "f"]],  d["y"], test_size=0.4, random_state=0)
#             prob = cl.predict_proba
            clf.fit(X_train, ycont_train) # i changed this to work with regression
            #score = clf.score(X_test, ycont_test) ## i removed this so that it worked with regression
            prob = clf.predict(X_test)
            
            lab_pred = prob.copy() # i added this to work with regression
            lab_pred=np.where(lab_pred >= 0.5, 1,-1)
            score = accuracy_score(lab_pred,y_test)
            rmse = mean_squared_error(ycont_test, prob)
#             accuracyM = scores.mean()
#             accuracyS = scores.std()
            accM.append(score)
            accS.append(rmse)
#             y1 = { 'y': yclass[val_idx].tolist()}
#             y2 = { 'y': yclass[train_idx].tolist()}
#             yf1 = pd.DataFrame(y1)
#             yf2 = pd.DataFrame(y2)
#             ylab = pd.concat([y1, y2]) 
            
#             y_pred_final = pd.DataFrame(d)
#             y_pred_final = pd.DataFrame(d)
#             y_pred_final["mean"] = y_pred_final.mean(axis=1)
#             y_pred_final["median"] = y_pred_final.iloc[:,0:4].median(axis=1)
#             y_pred_final['class'] = np.where(y_pred_final['mean'] >= 0.5, 1,-1)
# #             y_pred_final['class'] = np.where(y_pred_final['median'] >= 0.5, 1,-1)
            
#             rmse = mean_squared_error(ycont[val_idx], y_pred_final["mean"])
#             scores.append(rmse)
            
#             rmsemed = mean_squared_error(ycont[val_idx], y_pred_final["median"])
#             scoresM.append(rmsemed)
            
#             accuracy = accuracy_score(yclass[val_idx], y_pred_final["class"])
#             acc.append(accuracy)
    
#             rmse = mean_squared_error(y[val_idx], y_pred)
#             scores.append(rmse)

    mS = np.mean(accS)
    sS = np.std(accS)
    print("MSE: %0.3f (+/- %0.2f)" % (mS, sS))

#     return(d)
    mA = np.mean(accM)
    sA = np.std(accM)
    print("Accuracy: %0.3f (+/- %0.2f)" % (mA, sA))
    
#     mMed = np.mean(scoresM)
#     sMed = np.std(scoresM)
#     print("MSEmedian: %0.3f (+/- %0.2f)" % (mMed, sMed))

In [217]:
cv_idx

sklearn.cross_validation.KFold(n=2459, n_folds=2, shuffle=True, random_state=1)

In [220]:
# #only works with nd numpy arrays
# posPatternMMSE = normalize(posPatternClass)
# y = posPatternClass.truthClass.values
# X = posPatternMMSE.as_matrix(columns = posPatternMMSE.columns[1:len(posPatternMMSE.columns)-1])

# X_new = SelectKBest(f_classif, k="all").fit_transform(X, y)
for tdepth in [2,3,4,5,6,7,8,9,10,15,20]:
    print(tdepth)
    computeMixedAccuracyLog(posPatternClass, unigramsClass,  mixedClass, finalFeatures, finalFeaturesMSE,10,tdepth)

2
MSE: 0.045 (+/- 0.00)
Accuracy: 0.749 (+/- 0.03)
3
MSE: 0.041 (+/- 0.00)
Accuracy: 0.769 (+/- 0.03)
4
MSE: 0.039 (+/- 0.00)
Accuracy: 0.776 (+/- 0.03)
5
MSE: 0.039 (+/- 0.00)
Accuracy: 0.779 (+/- 0.03)
6
MSE: 0.039 (+/- 0.00)
Accuracy: 0.782 (+/- 0.03)
7
MSE: 0.039 (+/- 0.00)
Accuracy: 0.771 (+/- 0.02)
8
MSE: 0.040 (+/- 0.00)
Accuracy: 0.770 (+/- 0.02)
9
MSE: 0.040 (+/- 0.00)
Accuracy: 0.770 (+/- 0.02)
10
MSE: 0.040 (+/- 0.00)
Accuracy: 0.771 (+/- 0.02)
15
MSE: 0.041 (+/- 0.00)
Accuracy: 0.767 (+/- 0.02)
20
MSE: 0.042 (+/- 0.00)
Accuracy: 0.762 (+/- 0.02)


In [109]:
# ypredict = pd.DataFrame()
# ypredict["y1"] = computeMSE(posPatternM, 10, "f_Classif","all", "LinearSVR")
# ypredict["y2"] = computeMSE(unigramsM, 10, "f_Classif","all" ,"Log")
# ypredict["y3"] = computeMSE(mixedM, 10, "f_Classif","all", "LinearSVR")
# ypredict["y4"] = computeMSE(finalFeaturesM, 10, "f_Classif","all" ,"Log")

# y_pred = ypredict.mean(axis =1)
# rmse = mean_squared_error(finalFeaturesMSE.truthMean.values, y_pred.values)
computeMSE(posPatternM, 10, "f_Classif","all", "LinearSVR")

[[ 0.82751754 -1.33042654 -0.99268915 ... -0.24765905 -0.20373472
  -0.19714909]
 [ 0.82751754 -1.33042654  0.04339855 ... -0.24765905 -0.20373472
  -0.19714909]
 [-1.59062344 -0.34384063  0.04339855 ... -0.24765905 -0.20373472
  -0.19714909]
 ...
 [-0.98608819 -0.34384063 -0.99268915 ... -0.24765905 -0.20373472
  -0.19714909]
 [ 0.2229823   1.62933117 -0.99268915 ... -0.24765905 -0.20373472
  -0.19714909]
 [-0.38155295 -1.33042654  0.04339855 ...  3.92353005 -0.20373472
  -0.19714909]]
MSE: 0.054 (+/- 0.01)


In [194]:
allFeatures=pd.merge(posPatternClass[posPatternClass.columns[0:len(posPatternClass.columns)-1]],unigramsClass[unigramsClass.columns[0:len(unigramsClass.columns)-1]], on=["id"])
allFeatures=pd.merge(allFeatures,mixedClass[mixedClass.columns[0:len(mixedClass.columns)-1]], on=["id"])
allFeatures=pd.merge(allFeatures,finalFeaturesMSE, on=["id"])

In [196]:
computeMSE(allFeatures, 10, "featureself_classif", 250, "Log")

[[0.8275175413279654 -1.3304265357116412 -0.9926891489824088 ...
  4.374236300204959 0.2994538073821884 -1.3933368436270321]
 [0.8275175413279654 -1.3304265357116412 0.04339854938250764 ...
  -0.4353280201673156 -0.17838974378770206 0.7177015411412194]
 [-1.5906234380249364 -0.34384063362631984 0.04339854938250764 ...
  -0.4353280201673156 -0.6562332949575925 -1.3933368436270321]
 ...
 [-0.986088193186711 -0.34384063362631984 -0.9926891489824088 ...
  -0.4353280201673156 -0.6562332949575925 0.7177015411412194]
 [0.2229822964897399 1.6293311705443227 -0.9926891489824088 ...
  0.25175259702872366 0.7772973585520789 -1.3933368436270321]
 [-0.3815529483484855 -1.3304265357116412 0.04339854938250764 ...
  -0.4353280201673156 -0.6562332949575925 0.7177015411412194]]


ValueError: Unknown label type: 'continuous'

In [110]:
features2withLabel = pd.merge(features2, Labels[["id","truthClass"]], on = "id")
#Pick classifier for informality
classifiers = ["Log","SVM","RandomForest","XgBoost","NaiveBayes"]
for classif in classifiers:
    [acc1,acc2]=useNirmalFunctionEfficiently(classif,"f_classif","all",features2withLabel)
    print(classif+" yields accuracy of:"+str(acc1))


Log yields accuracy of:0.6945906779814721
SVM yields accuracy of:0.7031355862182929
RandomForest yields accuracy of:0.690955329895363
XgBoost yields accuracy of:0.7010964290920279
NaiveBayes yields accuracy of:0.6738621009774504


In [111]:
features1withLabel = pd.merge(features1, Labels[["id","truthClass"]], on = "id")
#Pick classifier for informality
classifiers = ["Log","SVM","RandomForest","XgBoost","NaiveBayes"]
for classif in classifiers:
    [acc1,acc2]=useNirmalFunctionEfficiently(classif,"f_classif","all",features1withLabel)
    print(classif+" yields accuracy of:"+str(acc1))


Log yields accuracy of:0.7031504921510423
SVM yields accuracy of:0.7059927022057959
RandomForest yields accuracy of:0.6881245192853481
XgBoost yields accuracy of:0.7019292938860149
NaiveBayes yields accuracy of:0.6738604552120049
